In [1]:
import os, sys
import pandas as pd

# Vectorized Feature Extraction
## Construct a DataFrame from the Raw Data Files

In [2]:
folders = [f for f in os.listdir('data') if not os.path.isfile(os.path.join('data', f)) and 
           f.find('_MODEL') < 0]

In [3]:
def parse_metadata_from_file_name(file_name):
    subject_id = file_name.split('-')[-1].split('.txt')[0]
    gender = 'Male' if subject_id[0] == 'm' else 'Female'
    timestamp = ''
    for i in file_name.split('-')[1:7]:
        timestamp += i
    return subject_id, gender, timestamp

In [4]:
data = pd.DataFrame()
for folder in folders:
    files = [f for f in os.listdir('data/'+folder) if 
             os.path.isfile(os.path.join('data/'+folder, f))]
    for file_name in files:
        raw_data = pd.read_csv('data/{0}/{1}'.format(folder, file_name), sep=' ', header=0)
        raw_data.columns = ['X', 'Y', 'Z']
        subject_id, gender, timestamp = parse_metadata_from_file_name(file_name)
        raw_data['SubjectID'] = subject_id
        raw_data['Gender'] = gender
        raw_data['Timestamp'] = timestamp
        raw_data['ADL'] = folder
        data = data.append(raw_data, ignore_index=True)
print data.shape

(445690, 7)


In [5]:
data.to_csv('data/adl_data.csv', sep=',', header=True, index=False)

## The Groupby Operation

The pandas groupby operation lets you apply a function to subsets of a DataFrame without using a for loop. This is very similar to applying an aggregate function with a group by clause in SQL, if you are familiar with the database language. If you are not, now is a great time to start experimenting with set-based logic.

For the [ADL data set](https://archive.ics.uci.edu/ml/datasets/Dataset+for+ADL+Recognition+with+Wrist-worn+Accelerometer), we want to calculate a set of features for every axis in every file. We will apply a groupby operation that aggregates by timestamp because it is unique to every file. Since we want the other indicators (subject_id and gender) and the label (ADL) to be in the output, and these are all guaranteed to be constant for a given timestamp, we will include those columns in the groupby specification. The order does not matter.

In [6]:
grouped_data = data.groupby(['SubjectID', 'Gender', 'Timestamp', 'ADL'])

Now we can easily generate a feature like the mean for every group in the set:

In [7]:
grouped_data[['X', 'Y', 'Z']].mean().head(20)

X          Y  \
SubjectID Gender Timestamp      ADL                                    
f1        Female 20110323103918 Standup_chair   16.760479  38.700599   
                 20110323104201 Standup_chair   18.016598  36.755187   
                 20110323104551 Standup_chair   19.256667  34.766667   
                 20110323104759 Standup_chair   18.295238  36.695238   
                 20110324094434 Comb_hair       31.016827  36.177885   
                 20110324094936 Standup_chair   15.892216  33.976048   
                 20110324095016 Sitdown_chair   23.974026  33.493506   
                 20110324095107 Walk            12.899059  36.757057   
                 20110324095211 Walk            12.460733  37.011518   
                 20110324100432 Pour_water      28.310260  40.707046   
                 20110324100702 Drink_glass     32.440285  39.655971   
                 20110324101602 Drink_glass     36.636475  38.585067   
                 20110324102402 Descend_stairs  17.202361  39.608769   
                 20110324102439 Climb_stairs    11.843866  36.736059   
                 20110324102511 Descend_stairs  14.954972  38.863039   
                 20110324102544 Climb_stairs    11.851301  36.795539   
                 20110324102633 Comb_hair       34.820652  35.990489   
                 20110324102733 Sitdown_chair   16.762082  36.910781   
                 20110324103817 Standup_chair   20.460177  34.823009   
                 20110324104625 Drink_glass     37.256983  39.607076   

                                                        Z  
SubjectID Gender Timestamp      ADL                        
f1        Female 20110323103918 Standup_chair   40.868263  
                 20110323104201 Standup_chair   40.937759  
                 20110323104551 Standup_chair   42.506667  
                 20110323104759 Standup_chair   40.701587  
                 20110324094434 Comb_hair       39.520433  
                 20110324094936 Standup_chair   38.952096  
                 20110324095016 Sitdown_chair   47.272727  
                 20110324095107 Walk            34.762190  
                 20110324095211 Walk            34.107853  
                 20110324100432 Pour_water      49.347342  
                 20110324100702 Drink_glass     48.552585  
                 20110324101602 Drink_glass     48.004896  
                 20110324102402 Descend_stairs  36.698145  
                 20110324102439 Climb_stairs    33.130112  
                 20110324102511 Descend_stairs  35.121951  
                 20110324102544 Climb_stairs    32.643123  
                 20110324102633 Comb_hair       40.646739  
                 20110324102733 Sitdown_chair   37.215613  
                 20110324103817 Standup_chair   41.902655  
                 20110324104625 Drink_glass     45.865922

With the agg() function, you can generate many different features in one pass through the original DataFrame. I will leave the verification of increased efficiency to you as an exercise.

In [8]:
from numpy import mean, std, min, max

features = grouped_data['X'].agg({
        'MeanX': mean,
        'StdX': std,
        'MinX': min,
        'MaxX': max,
        'LenX': len
    })

In [9]:
features.head(20)

MinX  MaxX  LenX      MeanX  \
SubjectID Gender Timestamp      ADL                                           
f1        Female 20110323103918 Standup_chair     11    32   167  16.760479   
                 20110323104201 Standup_chair      8    33   241  18.016598   
                 20110323104551 Standup_chair     10    36   300  19.256667   
                 20110323104759 Standup_chair      7    35   315  18.295238   
                 20110324094434 Comb_hair          6    63   832  31.016827   
                 20110324094936 Standup_chair      8    31   167  15.892216   
                 20110324095016 Sitdown_chair      8    31   154  23.974026   
                 20110324095107 Walk               0    38  1169  12.899059   
                 20110324095211 Walk               0    37   955  12.460733   
                 20110324100432 Pour_water        17    40   809  28.310260   
                 20110324100702 Drink_glass       18    43   561  32.440285   
                 20110324101602 Drink_glass       16    44   817  36.636475   
                 20110324102402 Descend_stairs     0    43   593  17.202361   
                 20110324102439 Climb_stairs       0    19   269  11.843866   
                 20110324102511 Descend_stairs     0    42   533  14.954972   
                 20110324102544 Climb_stairs       0    18   269  11.851301   
                 20110324102633 Comb_hair          6    63   736  34.820652   
                 20110324102733 Sitdown_chair      6    26   269  16.762082   
                 20110324103817 Standup_chair      8    31   339  20.460177   
                 20110324104625 Drink_glass       21    44   537  37.256983   

                                                     StdX  
SubjectID Gender Timestamp      ADL                        
f1        Female 20110323103918 Standup_chair    6.638887  
                 20110323104201 Standup_chair    8.690017  
                 20110323104551 Standup_chair    9.754724  
                 20110323104759 Standup_chair    9.104059  
                 20110324094434 Comb_hair       16.669570  
                 20110324094936 Standup_chair    6.802755  
                 20110324095016 Sitdown_chair    7.349312  
                 20110324095107 Walk             6.191721  
                 20110324095211 Walk             5.367352  
                 20110324100432 Pour_water       4.895584  
                 20110324100702 Drink_glass      7.170656  
                 20110324101602 Drink_glass      6.559777  
                 20110324102402 Descend_stairs  11.039067  
                 20110324102439 Climb_stairs     4.976603  
                 20110324102511 Descend_stairs   9.329084  
                 20110324102544 Climb_stairs     4.162758  
                 20110324102633 Comb_hair       15.451144  
                 20110324102733 Sitdown_chair    5.982803  
                 20110324103817 Standup_chair    8.462974  
                 20110324104625 Drink_glass      5.155155

## Define Feature Extraction Functions

Now, we would like to do this for every acclerometer axis with only a single pass through the DataFrame. Defining a set of functions will allow us to do that while also maintaining control over the named columns. This might seem like a lot of overhead for such readily available built-in functions like mean and std, but it will be become very useful when you write your own feature extraction algorithms.

In [10]:
def Length(data):
    """Returns the number of samples in a time series"""
    return len(data)

def Mean(data):
    """Returns the mean of a time series"""
    return data.mean()

def Std(data):
    """Returns the standard deviation a time series"""
    return data.std()

def Min(data):
    """Returns the mean of a time series"""
    return data.min()

def Max(data):
    """Returns the standard deviation a time series"""
    return data.max()

In [11]:
functions = [Mean, Std, Min, Max, Length]

features = grouped_data.agg({
        'X': functions, 
        'Y': functions,
        'Z': functions
    })

features.head(20)

Y                     \
                                                     Mean        Std Min Max   
SubjectID Gender Timestamp      ADL                                            
f1        Female 20110323103918 Standup_chair   38.700599   1.788568  35  46   
                 20110323104201 Standup_chair   36.755187   1.610585  29  41   
                 20110323104551 Standup_chair   34.766667   3.373944  30  41   
                 20110323104759 Standup_chair   36.695238   2.439551  27  43   
                 20110324094434 Comb_hair       36.177885  10.425466   0  63   
                 20110324094936 Standup_chair   33.976048   2.953356  23  37   
                 20110324095016 Sitdown_chair   33.493506   3.074769  22  44   
                 20110324095107 Walk            36.757057   2.735642  18  55   
                 20110324095211 Walk            37.011518   2.912216  29  54   
                 20110324100432 Pour_water      40.707046   4.723227  29  56   
                 20110324100702 Drink_glass     39.655971   6.836170  29  52   
                 20110324101602 Drink_glass     38.585067   8.369408  25  56   
                 20110324102402 Descend_stairs  39.608769   3.751854  27  56   
                 20110324102439 Climb_stairs    36.736059   2.087865  31  48   
                 20110324102511 Descend_stairs  38.863039   3.411888  29  55   
                 20110324102544 Climb_stairs    36.795539   2.038578  30  44   
                 20110324102633 Comb_hair       35.990489  11.661725   0  63   
                 20110324102733 Sitdown_chair   36.910781   6.595979  18  55   
                 20110324103817 Standup_chair   34.823009   2.349229  22  41   
                 20110324104625 Drink_glass     39.607076  11.206183  18  57   

                                                               X             \
                                               Length       Mean        Std   
SubjectID Gender Timestamp      ADL                                           
f1        Female 20110323103918 Standup_chair     167  16.760479   6.638887   
                 20110323104201 Standup_chair     241  18.016598   8.690017   
                 20110323104551 Standup_chair     300  19.256667   9.754724   
                 20110323104759 Standup_chair     315  18.295238   9.104059   
                 20110324094434 Comb_hair         832  31.016827  16.669570   
                 20110324094936 Standup_chair     167  15.892216   6.802755   
                 20110324095016 Sitdown_chair     154  23.974026   7.349312   
                 20110324095107 Walk             1169  12.899059   6.191721   
                 20110324095211 Walk              955  12.460733   5.367352   
                 20110324100432 Pour_water        809  28.310260   4.895584   
                 20110324100702 Drink_glass       561  32.440285   7.170656   
                 20110324101602 Drink_glass       817  36.636475   6.559777   
                 20110324102402 Descend_stairs    593  17.202361  11.039067   
                 20110324102439 Climb_stairs      269  11.843866   4.976603   
                 20110324102511 Descend_stairs    533  14.954972   9.329084   
                 20110324102544 Climb_stairs      269  11.851301   4.162758   
                 20110324102633 Comb_hair         736  34.820652  15.451144   
                 20110324102733 Sitdown_chair     269  16.762082   5.982803   
                 20110324103817 Standup_chair     339  20.460177   8.462974   
                 20110324104625 Drink_glass       537  37.256983   5.155155   

                                                                       Z  \
                                               Min Max Length       Mean   
SubjectID Gender Timestamp      ADL                                        
f1        Female 20110323103918 Standup_chair   11  32    167  40.868263   
                 20110323104201 Standup_chair    8  33    241  40.937759   
                 

## Put the Functions in a Module and Apply to the DataFrame

If you find yourself repeatedly analyzing time series data, e.g. for wearable sensor application development, this technique will come in handy over and over again. I have a feature function library (features.py) that you are welcome to use and add to: https://github.com/mhbuehler/feature_extraction

This is how you import and apply the functions:

In [12]:
from features import *

functions = [Mean, Median, Std, IQR, Length, Min, Max, DominantFrequency]

In [13]:
features = grouped_data.agg({
        'X': functions, 
        'Y': functions,
        'Z': functions
    })

features.head(20)

Y                    \
                                                     Mean Median        Std   
SubjectID Gender Timestamp      ADL                                           
f1        Female 20110323103918 Standup_chair   38.700599   38.0   1.788568   
                 20110323104201 Standup_chair   36.755187   37.0   1.610585   
                 20110323104551 Standup_chair   34.766667   33.5   3.373944   
                 20110323104759 Standup_chair   36.695238   36.0   2.439551   
                 20110324094434 Comb_hair       36.177885   36.0  10.425466   
                 20110324094936 Standup_chair   33.976048   35.0   2.953356   
                 20110324095016 Sitdown_chair   33.493506   33.0   3.074769   
                 20110324095107 Walk            36.757057   36.0   2.735642   
                 20110324095211 Walk            37.011518   36.0   2.912216   
                 20110324100432 Pour_water      40.707046   41.0   4.723227   
                 20110324100702 Drink_glass     39.655971   40.0   6.836170   
                 20110324101602 Drink_glass     38.585067   35.0   8.369408   
                 20110324102402 Descend_stairs  39.608769   39.0   3.751854   
                 20110324102439 Climb_stairs    36.736059   37.0   2.087865   
                 20110324102511 Descend_stairs  38.863039   38.0   3.411888   
                 20110324102544 Climb_stairs    36.795539   37.0   2.038578   
                 20110324102633 Comb_hair       35.990489   37.5  11.661725   
                 20110324102733 Sitdown_chair   36.910781   37.0   6.595979   
                 20110324103817 Standup_chair   34.823009   35.0   2.349229   
                 20110324104625 Drink_glass     39.607076   45.0  11.206183   

                                                                     \
                                                 IQR Length Min Max   
SubjectID Gender Timestamp      ADL                                   
f1        Female 20110323103918 Standup_chair    1.0    167  35  46   
                 20110323104201 Standup_chair    2.0    241  29  41   
                 20110323104551 Standup_chair    7.0    300  30  41   
                 20110323104759 Standup_chair    2.0    315  27  43   
                 20110324094434 Comb_hair        7.5    832   0  63   
                 20110324094936 Standup_chair    3.0    167  23  37   
                 20110324095016 Sitdown_chair    1.0    154  22  44   
                 20110324095107 Walk             2.0   1169  18  55   
                 20110324095211 Walk             3.0    955  29  54   
                 20110324100432 Pour_water       6.0    809  29  56   
                 20110324100702 Drink_glass     14.0    561  29  52   
                 20110324101602 Drink_glass     16.0    817  25  56   
                 20110324102402 Descend_stairs   3.0    593  27  56   
                 20110324102439 Climb_stairs     2.0    269  31  48   
                 20110324102511 Descend_stairs   3.0    533  29  55   
                 20110324102544 Climb_stairs     3.0    269  30  44   
                 20110324102633 Comb_hair       14.0    736   0  63   
                 20110324102733 Sitdown_chair   10.0    269  18  55   
                 20110324103817 Standup_chair    2.0    339  22  41   
                 20110324104625 Drink_glass     21.0    537  18  57   

                                                                          X  \
                                               DominantFrequency       Mean   
SubjectID Gender Timestamp      ADL                                           
f1        Female 20110323103918 Standup_chair                  0  16.760479   
                 20110323104201 Standup_chair                  0  18.016598   
                 20110323104551 Standup_chair                  0  19.256667   
                 20110323104759 Standup_chair                  0  18.295238   
                 20110324094434 Comb_hair          

## Create the Flat Feature File
Now let's reformat the index and column names and save it to CSV. This file will be the source of training examples for our machine learning models.

In [14]:
features.reset_index(inplace=True)
features.columns = ['{0}{1}'.format(a, b) if b else a for a, b in features.columns]

features.head(20)

,SubjectID,Gender,Timestamp,ADL,YMean,YMedian,YStd,YIQR,YLength,YMin,...,XMax,XDominantFrequency,ZMean,ZMedian,ZStd,ZIQR,ZLength,ZMin,ZMax,ZDominantFrequency
0,f1,Female,20110323103918,Standup_chair,38.700599,38.0,1.788568,1.0,167,35,...,32,0,40.868263,38.0,5.888063,9.0,167,35,53,0
1,f1,Female,20110323104201,Standup_chair,36.755187,37.0,1.610585,2.0,241,29,...,33,0,40.937759,35.0,8.061241,17.0,241,32,58,0
2,f1,Female,20110323104551,Standup_chair,34.766667,33.5,3.373944,7.0,300,30,...,36,0,42.506667,38.0,6.873191,14.0,300,35,55,0
3,f1,Female,20110323104759,Standup_chair,36.695238,36.0,2.439551,2.0,315,27,...,35,0,40.701587,35.0,7.923992,17.0,315,32,54,0
4,f1,Female,20110324094434,Comb_hair,36.177885,36.0,10.425466,7.5,832,0,...,63,0,39.520433,38.0,7.095333,11.0,832,19,61,0
5,f1,Female,20110324094936,Standup_chair,33.976048,35.0,2.953356,3.0,167,23,...,31,0,38.952096,35.0,8.651769,15.0,167,24,54,0
6,f1,Female,20110324095016,Sitdown_chair,33.493506,33.0,3.074769,1.0,154,22,...,31,0,47.272727,52.0,8.769580,10.0,154,24,61,0
7,f1,Female,20110324095107,Walk,36.757057,36.0,2.735642,2.0,1169,18,...,38,0,34.762190,34.0,4.277799,3.0,1169,27,52,0
8,f1,Female,20110324095211,Walk,37.011518,36.0,2.912216,3.0,955,29,...,37,0,34.107853,34.0,3.539814,3.0,955,27,53,0
9,f1,Female,20110324100432,Pour_water,40.707046,41.0,4.723227,6.0,809,29,...,40,0,49.347342,50.0,2.324375,3.0,809,38,61,0


In [15]:
features.to_csv('data/features.csv', index=False)